## Treinamento do modelo

Esse notebook está responsável por treinar o modelo a partir dos dados adquiridos no banco de dados. Para garantir a entrega da tarefa no tempo solicitado, o escopo do modelo foi mantido de forma mais simples, com muitas possibilidades de melhoria.

---


In [1]:
# importando pacotes
import os
import pandas as pd
import numpy as np

import datetime
import holidays

import xgboost as xgb

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from sklearn.preprocessing import LabelEncoder

In [2]:
# agora que temos os dados preparados, podemos começar a treinar o modelo
pasta_atual = os.getcwd()
pasta_pai = os.path.dirname(pasta_atual)
pasta_data = os.path.join(pasta_pai, "data")

df = pd.read_csv(f"{pasta_data}\\dados_limpos_e_completos.csv", parse_dates=["date"])\
    .sort_values(by=["date", "appId"])\
    .reset_index(drop=True)

In [3]:
# definir dados de treinamento e de validação
df["date"].describe()

count                            54795
mean     2024-05-20 23:59:59.999999744
min                2024-01-02 00:00:00
25%                2024-03-12 00:00:00
50%                2024-05-21 00:00:00
75%                2024-07-30 00:00:00
max                2024-10-08 00:00:00
Name: date, dtype: object

In [4]:
# Vamos utilizar todos os dados disponíveis, exceto os últimos 30 dias para fazer o treinamento do modelo
# Os 30 últimos dados vão ser utilizados na etapa seguinte
data_minima_treinamento = df["date"].min()
data_maxima_treinamento = df["date"].max() - pd.Timedelta(days=30)

In [5]:
print(f"minimo: {data_minima_treinamento} - maximo: {data_maxima_treinamento}")

minimo: 2024-01-02 00:00:00 - maximo: 2024-09-08 00:00:00


In [6]:
df_treinamento = df.loc[
    (df["date"] >= data_minima_treinamento)
    & (df["date"] <= data_maxima_treinamento)
]

In [7]:
# Vamos desconsiderar as colunas de "appId", "date" e "proximo_dauReal" nos dados de treinamento
colunas_X = [col for col in df.columns if col not in ["appId", "date", "proximo_dauReal"]]
colunas_y = "proximo_dauReal"

In [8]:
# Vamos treinar o modelo e já calcular algumas métricas dos dados de teste
def treinar_avaliar_modelo(aux_X, aux_y):
    X = aux_X.copy()
    y = aux_y.copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

    # idealmente, a escolha dos hyperparâmetros é feito através de um processo estruturado 
    model = xgb.XGBRegressor(
        objective = 'count:poisson', # específico para contagem de eventos
        n_estimators = 1200,
        learning_rate = 0.2,
        max_depth = 6,
        random_state = 42
    )

    print(f"Treinando modelo...\n")

    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)
    
    y_pred = model.predict(X_test)
    
    msle  = mean_squared_log_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    mae  = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"MSLE: {msle :.4f}")
    print(f"MAPE: {mape :.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R2: {r2:.4f}\n")

    return model

X = df_treinamento[colunas_X]
y = df_treinamento[colunas_y]

model = treinar_avaliar_modelo(X, y)

Treinando modelo...

MSLE: 0.0242
MAPE: 0.1262
MAE: 20122.3173
R2: 0.9943



In [9]:
# Devido a grande variedade dos dados, os resultados se mostraram satisfatórios para o escopo atual do problema.
# As possíveis melhorias vão ser descritas no arquivo README.md da apresentação do projeto.

# Com o modelo treinado, podemos armazenar ele em um arquivo
pasta_atual = os.getcwd()
pasta_pai = os.path.dirname(pasta_atual)
pasta_models = os.path.join(pasta_pai, "models")

model.save_model(f"{pasta_models}\\xgb_model.json")